In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import statsmodels.formula.api as smf
import statsmodels.stats.anova as anova

In [2]:
Data=pd.read_excel("https://data.hossam.kr/E02/store.xlsx")
data=pd.DataFrame(Data)

In [3]:
data.head(2)

,store1,store2
0,46,78
1,47,57


## 분산분석 전제조건

1. 독립성
2. 정규성

### 1. 데이터의 독립성 테스트(카이제곱)

In [13]:
sp.stats.chi2_contingency(data[["store1","store2"]])

Chi2ContingencyResult(statistic=64.44306604494015, pvalue=1.8512336438945882e-10, dof=9, expected_freq=array([[56.38709677, 67.61290323],
       [47.29240375, 56.70759625],
       [40.47138398, 48.52861602],
       [34.10509886, 40.89490114],
       [42.74505723, 51.25494277],
       [61.38917794, 73.61082206],
       [41.83558793, 50.16441207],
       [37.74297607, 45.25702393],
       [37.74297607, 45.25702393],
       [37.28824142, 44.71175858]]))

### 2. 데이터의 정규성 테스트

#### 1) Shapiro wilk 검정
> 샘플 수가 적을 떄 (50개 미만, 정규분포성은 30미만을 권장)

|$H_0$|집단간 데이터 분포에는 차이가 없다|
|--|--|
|$H_1$|집단간 데이터 분포에는 차이가 있다|

In [4]:
sp.stats.shapiro(data["store1"])

ShapiroResult(statistic=0.8321117162704468, pvalue=0.035477906465530396)

In [5]:
sp.stats.shapiro(data["store2"])

ShapiroResult(statistic=0.8993193507194519, pvalue=0.21535511314868927)

#### 2) Nomal_Test 검정
> 표본의 수가 20개 이상이여야 유효

In [6]:
sp.stats.normaltest(data["store1"])

c:\Users\SAMSUNG\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


NormaltestResult(statistic=1.9891717867491527, pvalue=0.369876581177278)

#### 3) Kolmogorov-Smirnov Test
> 두 표본이 같은 분포를 따르는지를 Test

In [7]:
sp.stats.ks_2samp(data[data.columns[0]],data[data.columns[1]])

KstestResult(statistic=0.4, pvalue=0.41752365281777043, statistic_location=56, statistic_sign=1)

### 3.데이터의 등분산성

#### 1) Bartlett 검정
>집단간 분산이 같은지를 검사<br>
>모든 변수가 정규분포를 따른다는 가정이 필요

In [8]:
sp.stats.bartlett(data["store1"],data["store2"])

BartlettResult(statistic=1.0488412011085946, pvalue=0.305774119649436)

#### 2) fligner 검정
>비모수 등분산성 검사<br>
>각 독립표본들이 정규분포를 따르지 않아도 사용하는 검정방법

In [9]:
sp.stats.fligner(data["store1"],data["store2"])

FlignerResult(statistic=1.3014081560908837, pvalue=0.2539561678380817)

#### 3) Levene's test
> 레빈검정은 정규성 가정을 거의 하지 않음

In [10]:
sp.stats.levene(data["store1"],data["store2"])

LeveneResult(statistic=1.333315753388535, pvalue=0.2633161881599037)

## statesmodel을 이용한 F-oneway(F-test)

In [17]:
data2=data.melt().rename(columns=dict(zip(data.melt().columns,["구분","값"])))
data2["구분"]=data2["구분"].astype("category")

In [21]:
model=smf.ols("값 ~ C(구분)",data=data2).fit()

In [25]:
result=anova.anova_lm(model)
print(result)

            df   sum_sq     mean_sq         F    PR(>F)
C(구분)      1.0   378.45  378.450000  1.459162  0.242696
Residual  18.0  4668.50  259.361111       NaN       NaN


In [28]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      값   R-squared:                       0.075
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     1.459
Date:                Wed, 19 Jul 2023   Prob (F-statistic):              0.243
Time:                        16:01:46   Log-Likelihood:                -82.907
No. Observations:                  20   AIC:                             169.8
Df Residuals:                      18   BIC:                             171.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          43.7000      5.093      8.581      0.000      33.001      54.399
C(구분)[T.store2]     8.7000      7.202      1.208      0.243      -6.431      23.831
==============================================================================
Omnibus:                        2.943   Durbin-Watson:                   2.388
Prob(Omnibus):                  0.230   Jarque-Bera (JB):                1.241
Skew:                          -0.098   Prob(JB):                        0.538
Kurtosis:                       1.795   Cond. No.                         2.62
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""